# Collatz Proof

In this version, we tweak the function so that for odd we also divide by 2 once.

In this version we are discovering the generator function for "the next bits"

## Predictive Power

In [130]:
import sys, io
import math
import numpy as np
import pandas as pd
from scipy.optimize import nnls

pd.set_option('display.max_rows', None)

In [20]:
def collatzNext(n):
    if n %2 == 0:
        return n//2
    return (3*n + 1)//2

def collatzIter(start, iterations):
    n = start
    count = 0
    while count < iterations:
        if n %2 == 0:
            m = n//2
            yield n
            n = m
        else:
            m = (3*n + 1)//2
            yield n
            n = m
        count = count + 1
    #
    return 1


In [4]:
def blocks(sequence):
    for n in sequence:
        if  n %2 == 0:
            yield '□'
        else:
            yield '■'
    

In [5]:
def split(q, L_in):
    L_out = []
    for elem in L_in:
        L_out.append(elem)
        L_out.append(elem + 2**q)
    q = q + 1
    return q, L_out
#
def gen_partitions():
    L = [1]
    for p in range(1,8,1):
        q, L = split(p, L)
        print("%d   %s"%(q, L))
#
gen_partitions()

2   [1, 3]
3   [1, 5, 3, 7]
4   [1, 9, 5, 13, 3, 11, 7, 15]
5   [1, 17, 9, 25, 5, 21, 13, 29, 3, 19, 11, 27, 7, 23, 15, 31]
6   [1, 33, 17, 49, 9, 41, 25, 57, 5, 37, 21, 53, 13, 45, 29, 61, 3, 35, 19, 51, 11, 43, 27, 59, 7, 39, 23, 55, 15, 47, 31, 63]
7   [1, 65, 33, 97, 17, 81, 49, 113, 9, 73, 41, 105, 25, 89, 57, 121, 5, 69, 37, 101, 21, 85, 53, 117, 13, 77, 45, 109, 29, 93, 61, 125, 3, 67, 35, 99, 19, 83, 51, 115, 11, 75, 43, 107, 27, 91, 59, 123, 7, 71, 39, 103, 23, 87, 55, 119, 15, 79, 47, 111, 31, 95, 63, 127]
8   [1, 129, 65, 193, 33, 161, 97, 225, 17, 145, 81, 209, 49, 177, 113, 241, 9, 137, 73, 201, 41, 169, 105, 233, 25, 153, 89, 217, 57, 185, 121, 249, 5, 133, 69, 197, 37, 165, 101, 229, 21, 149, 85, 213, 53, 181, 117, 245, 13, 141, 77, 205, 45, 173, 109, 237, 29, 157, 93, 221, 61, 189, 125, 253, 3, 131, 67, 195, 35, 163, 99, 227, 19, 147, 83, 211, 51, 179, 115, 243, 11, 139, 75, 203, 43, 171, 107, 235, 27, 155, 91, 219, 59, 187, 123, 251, 7, 135, 71, 199, 39, 167, 103, 231,

In [14]:
def next_bits(p, L):
    B = []
    for elem in L:
        x = 2**p + elem
        for i in range(p-1):
            x = collatzNext(x)
        if x % 2 == 0:
            bit = 0
        else:
            bit = 1
        B.append(bit)
        
        if elem == 5:
            print("next bit for 5 is %d"%(bit))
    return B

def numFromBits(A):
    B = list(reversed(A))
    s = "".join(map(str, B))
    n = int(s, base=2)
    return n
#

def gen_partition_next_bit():
    L = [1]
    for p in range(1,8,1):
        q, L = split(p, L)
        print("%d   %s"%(q, L))
        B = next_bits(q, L)
        print("%d   %s"%(q, B))
        n = numFromBits(B)
        print("%d   %d"%(q,n))
#
gen_partition_next_bit()

2   [1, 3]
2   [0, 1]
2   2
3   [1, 5, 3, 7]
next bit for 5 is 0
3   [1, 0, 0, 1]
3   9
4   [1, 9, 5, 13, 3, 11, 7, 15]
next bit for 5 is 0
4   [0, 1, 0, 1, 0, 1, 0, 1]
4   170
5   [1, 17, 9, 25, 5, 21, 13, 29, 3, 19, 11, 27, 7, 23, 15, 31]
next bit for 5 is 1
5   [1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1]
5   39573
6   [1, 33, 17, 49, 9, 41, 25, 57, 5, 37, 21, 53, 13, 45, 29, 61, 3, 35, 19, 51, 11, 43, 27, 59, 7, 39, 23, 55, 15, 47, 31, 63]
next bit for 5 is 0
6   [0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1]
6   2858789542
7   [1, 65, 33, 97, 17, 81, 49, 113, 9, 73, 41, 105, 25, 89, 57, 121, 5, 69, 37, 101, 21, 85, 53, 117, 13, 77, 45, 109, 29, 93, 61, 125, 3, 67, 35, 99, 19, 83, 51, 115, 11, 75, 43, 107, 27, 91, 59, 123, 7, 71, 39, 103, 23, 87, 55, 119, 15, 79, 47, 111, 31, 95, 63, 127]
next bit for 5 is 1
7   [1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 

In [26]:
''.join(list(blocks(collatzIter(15, 80))))

'■■■■□□□□■□□□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□'

In [27]:
''.join(list(blocks(collatzIter(31, 80))))

'■■■■■□■□■■□■■■□■■■■□■□□■■■□■■□■■■■■■□□■■■■□□□■□■□■□□□■□□■■■□□□□■□□□■□■□■□■□■□■□■'

In [25]:
''.join(list(blocks(collatzIter(47, 80))))

'■■■■□■□■■□■■■□■■■■□■□□■■■□■■□■■■■■■□□■■■■□□□■□■□■□□□■□□■■■□□□□■□□□■□■□■□■□■□■□■□'

In [28]:
''.join(list(blocks(collatzIter(63, 80))))

'■■■■■■□□□■■□■■■□■■■■□■□□■■■□■■□■■■■■■□□■■■■□□□■□■□■□□□■□□■■■□□□□■□□□■□■□■□■□■□■□'

In [29]:
''.join(list(blocks(collatzIter(79, 80))))

'■■■■□□■□□□■■□□■■□■□□■□□□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□'

In [30]:
len('■■■■■□■□■■□■■■□■■■■□■□□■■■□■■□■■■■■■□□■■■■□□□■□■□■□□□■□□■■■□□□□■□□□■□')

69

In [34]:
print(list(collatzIter(31, 10)))

[31, 47, 71, 107, 161, 242, 121, 182, 91, 137]


In [35]:
print(list(collatzIter(47, 9)))

[47, 71, 107, 161, 242, 121, 182, 91, 137]


In [36]:
print(list(collatzIter(63, 11)))

[63, 95, 143, 215, 323, 485, 728, 364, 182, 91, 137]


In [121]:
print(list(collatzIter(15, len('■■■■□□□□'))))

[15, 23, 35, 53, 80, 40, 20, 10]


In [126]:
print(list(collatzIter(79, len('■■■■□□■□□□■■□□■■□■□□'))))

[79, 119, 179, 269, 404, 202, 101, 152, 76, 38, 19, 29, 44, 22, 11, 17, 26, 13, 20, 10]


In [125]:
print(list(collatzIter(69, 11)))

[69, 104, 52, 26, 13, 20, 10, 5, 8, 4, 2]


In [38]:
print(list(collatzIter(128+5, 6)))

[133, 200, 100, 50, 25, 38]


In [66]:
A = np.array([
    [-3,2,0,0,0,0,0],
    [0,-1,2,0,0,0,0],
    [0,0,-1,2,0,0,0],
    [0,0,0,-1,2,0,0],
    [0,0,0,0,-1,2,0],
    [0,0,0,0,0,-3,2]
])
b = np.array([1,0,0,0,0,1])
x, rnorm = nnls(A,b)

In [67]:
x

array([0.      , 0.5     , 0.25    , 0.125   , 0.0625  , 0.03125 ,
       0.546875])

In [69]:
x*64

array([ 0., 32., 16.,  8.,  4.,  2., 35.])

In [70]:
rnorm

0.0

# This representation of 5,8,4,2,1,2,1 works

In [101]:
A = np.array([
    [-3,2,0,0,0,0,0],
    [0,-1,2,0,0,0,0],
    [0,0,-1,2,0,0,0],
    [0,0,0,-1,2,0,0],
    [0,0,0,0,-1,2,0],
    [0,0,0,0,0,-3,2],
    [0,0,0,0,1, 0,0]
])
b = np.array([1,0,0,0,0,1,1])
np.linalg.det(A)

12.0

In [102]:
x

array([0.      , 0.5     , 0.25    , 0.125   , 0.0625  , 0.03125 ,
       0.546875])

In [103]:
A0 = A.copy()
A0.T[0] = b
A0

array([[ 1,  2,  0,  0,  0,  0,  0],
       [ 0, -1,  2,  0,  0,  0,  0],
       [ 0,  0, -1,  2,  0,  0,  0],
       [ 0,  0,  0, -1,  2,  0,  0],
       [ 0,  0,  0,  0, -1,  2,  0],
       [ 1,  0,  0,  0,  0, -3,  2],
       [ 1,  0,  0,  0,  1,  0,  0]])

In [104]:
np.linalg.det(A0)/np.linalg.det(A)

4.999999999999999

The sequence which imposes a cycle without explicitly specifying that the sequence contains a $1$

  -  $x_{1}$  = $(3x_{0}+1)/2$
    
  -  $x_{2}$  = $x_{1}/2$
    
  -  $x_{3}$  = $x_{2}/2$
    
  -  $x_{4}$  = $x_{3}/2$
    
  -  $x_{5}$ = $(3x_{4}+1)/2$

  -  $x_{6}$ = $x_{5}/2$

  -  $x_{7}$ = $x_{5}$

 Imposes the same constraints as the sequence that explicitly terminates at $1$ two steps earlier:

  -  $x_{1}$  = $(3x_{0}+1)/2$
    
  -  $x_{2}$  = $x_{1}/2$
    
  -  $x_{3}$  = $x_{2}/2$
    
  -  $x_{4}$  = $x_{3}/2$
    
  -  $x_{5}$ = $1$



In [118]:
A = np.array([
    [-3,2,0,0,0,0,0],
    [0,-1,2,0,0,0,0],
    [0,0,-1,2,0,0,0],
    [0,0,0,-1,2,0,0],
    [0,0,0,0,-3,2,0],
    [0,0,0,0,0,-1,2],
    [0,0,0,0,-1,0, 1],
])
b = np.array([1,0,0,0,1,0,0])
d = np.linalg.det(A)
A0 = A.copy()
A0.T[0] = b
d0 = np.linalg.det(A0)
(d0, d, d0/d)

(-15.0, -2.9999999999999996, 5.000000000000001)

In [120]:
A = np.array([
    [-3,2,0,0,0],
    [0,-1,2,0,0],
    [0,0,-1,2,0],
    [0,0,0,-1,2],
    [0,0,0,0, 1]
    
])
b = np.array([1,0,0,0,1])
d = np.linalg.det(A)
A0 = A.copy()
A0.T[0] = b
d0 = np.linalg.det(A0)
(d0, d, d0/d)

(14.999999999999993, 3.0000000000000004, 4.999999999999997)

In [119]:
A = np.array([
    [-3,2,0,0,0,0,0],
    [0,-1,2,0,0,0,0],
    [0,0,-1,2,0,0,0],
    [0,0,0,-1,2,0,0],
    [0,0,0,0,-3,2,0],
    [0,0,0,0,0,-1,2],
    [0,0,0,0,0,0, 1],
])
b = np.array([1,0,0,0,1,0,1])
d = np.linalg.det(A)
A0 = A.copy()
A0.T[0] = b
d0 = np.linalg.det(A0)
(d0, d, d0/d)

(44.99999999999997, 9.000000000000002, 4.999999999999996)

In [127]:
len('■■■■■□■□■■□■■■□■■■■□■□□■■■□■■□■■■■■■□□■■■■□□□■□■□■□□□■□□■■■□□□□■□□□■□■'), len('□□□□□□□□□□□□□□□□□□□□□□□□□□□□□')

(70, 29)

In [131]:
(69 * math.log(3) / math.log(2) ) - 28

81.3624125497598

In [135]:
x = 2**69 + 31
for i in range(69):
    x = collatzNext(x)
print(x)
print(math.log(x,2.0))

12157665459056928802
63.39850002884625


In [136]:
12157665459056928802 - 2**63

2934293422202152994

In [137]:
print(math.log(2934293422202152994, 2.0))

61.347718852190994


In [138]:
2934293422202152994 - 2**61

628450412988459042

In [139]:
print(math.log(628450412988459042, 2.0))

59.12457652836275


In [140]:
628450412988459042 - 2**59

51989660685035554

In [141]:
print(math.log(51989660685035554, 2.0))

55.52907425757527


In [142]:
51989660685035554 - 2**55

15960863666071586

In [143]:
print(math.log(15960863666071586, 2.0))

53.82538823824467


In [144]:
15960863666071586 - 2**53

6953664411330594

In [145]:
print(math.log(6953664411330594, 2.0))

52.6266948665923


In [146]:
6953664411330594 - 2**52

2450064783960098

In [147]:
print(math.log(2450064783960098, 2.0))

51.121741320400446


In [148]:
2450064783960098 - 2**51

198264970274850

In [149]:
print(math.log(198264970274850, 2.0))

47.49442313117902


In [150]:
198264970274850 - 2**47

57527481919522

In [151]:
print(math.log(57527481919522, 2.0))

45.70931655580507


In [152]:
57527481919522 - 2**45

22343109830690

In [153]:
print(math.log(22343109830690, 2.0))

44.34489523516133


In [154]:
22343109830690 - 2**44

4750923786274

In [155]:
print(math.log(4750923786274, 2.0))

42.11134520204908


In [156]:
4750923786274 - 2**42

352877275170

In [157]:
print(math.log(352877275170, 2.0))

38.36037556921674


In [158]:
352877275170 - 2**38

77999368226

In [159]:
print(math.log(77999368226, 2.0))

36.182743387452255


In [160]:
77999368226 - 2**36

9279891490

In [161]:
print(math.log(9279891490, 2.0))

33.11146078996905


In [164]:
9279891490 - 2**33

689956898

In [165]:
print(math.log(689956898, 2.0))

29.36193099782461


In [166]:
689956898 - 2**29

153085986

In [167]:
print(math.log(153085986, 2.0))

27.189766978881845


In [168]:
153085986 - 2**27

18868258

In [169]:
print(math.log(18868258, 2.0))

24.169457897389893


In [170]:
18868258 - 2**24

2091042

In [171]:
print(math.log(2091042, 2.0))

20.995790609053287


In [172]:
2091042 - 2**20

1042466

In [173]:
print(math.log(1042466, 2.0))

19.99156890030223


In [174]:
1042466 - 2**19

518178

In [175]:
print(math.log(518178, 2.0))

18.98308823954058


In [176]:
518178 - 2**18

256034

In [177]:
print(math.log(256034, 2.0))

17.965975879874367


In [178]:
256034 - 2**17

124962

In [179]:
print(math.log(124962, 2.0))

16.931129923354177


In [180]:
124962 - 2**16

59426

In [181]:
print(math.log(59426, 2.0))

15.858806655090357


In [182]:
59426 - 2**15

26658

In [183]:
print(math.log(26658, 2.0))

14.70228092673116


In [184]:
26658 - 2**14

10274

In [185]:
print(math.log(10274, 2.0))

13.32671035834696


In [186]:
10274 - 2**13

2082

In [187]:
print(math.log(2082, 2.0))

11.023754353299417


In [188]:
2082 - 2**11

34

In [189]:
print(math.log(34, 2.0))

5.08746284125034


In [190]:
34 - 2**5

2